In [11]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def conv_2d(input_data,k_h=3,k_w=3,s_h=1,s_w=1,
            output_channel=64,padding='SAME',name='conv_i'):
    
    input_data_channel=int(input_data.shape[-1])
    # 这个可以理解为是方便显示用的
    with tf.name_scope(name) as scope:
        initial_W = tf.truncated_normal(shape=[k_h,k_w,
                                               input_data_channel,
                                               output_channel],
                                        stddev=0.1)
        W=tf.Variable(initial_W,name=name+'_'+'weights')
        
        initial_b = tf.constant(0.1,shape=[output_channel])
        b=tf.Variable(initial_b,name=name+'_'+'bias')
        '''
        strids表示步长的维度信息，定义如下：
                [batch, height, width, channels]维度上的步长
        '''
        conv_op=tf.nn.conv2d(input_data,W,strides=[1,s_h,s_w,1],
                             padding=padding,
                             name=name+'_'+'conv')
        
        result=conv_op+b
        
        # 默认激活函数是relu
        result_act=tf.nn.relu(result,name=name+'_'+'activation')
        return result_act

In [4]:
'''
定义池化层
'''
def max_pool_2d(input_data,p_h=2,p_w=2,s_h=2,s_w=2,
                padding='SAME',name='pool_i'):
    pool_size=[1,p_h,p_w,1]
    pool_strid=[1,s_h,s_w,1]
    with tf.name_scope(name) as scope:
        max_poolled=tf.nn.max_pool(input_data,
                                   ksize=pool_size,
                                   strides=pool_strid,
                                  padding=padding,name=name+'_maxpool')
    
    return max_poolled

In [5]:
'''
定义全连接层
'''
def dense_2d(input_data,output_dim,keep_prob=0.9,name='dense_i',act=tf.nn.relu):
    input_dim=int(input_data.shape[-1])
    with tf.name_scope(name) as scope:
        # 相当于通过这个方式定义了随机初始化的格式
        # 可以理解为只有被定义为/被包裹为variable的tensor，
        # 才能够被用来被梯度下降学习
        W=tf.truncated_normal(shape=[input_dim,output_dim],
                                        stddev=0.1)
        W=tf.Variable(W,name=name+'_weights')
        
        b=tf.constant(value=0.1,shape=[output_dim])
        b=tf.Variable(b,name=name+'_bias')
        
        dense=tf.matmul(input_data,W)+b
        dense=act(dense)
            
        dense_dropped=tf.nn.dropout(dense,keep_prob=keep_prob)
    
    return dense_dropped

In [22]:
'''
之前出问题，因该是有一下原因：
    1、maxpool的步长不对，应该是2的，我写成了1
    2、最后全连接层名字不对，用了同一个名字，应该导致参数共享了。
'''
tf.reset_default_graph()
'''
定义输入
'''
x_input=tf.placeholder(shape=(None,784),dtype=tf.float32,name='x_input')
y_input=tf.placeholder(shape=(None,10),dtype=tf.float32,name='y_input')
keep_prob=tf.placeholder(dtype=tf.float32,name='keep_prob')

x_input_reshaped=tf.reshape(x_input,shape=[-1,28,28,1])

h_conv1=conv_2d(x_input_reshaped,
                k_h=5,k_w=5,
                s_h=1,s_w=1,
                output_channel=32,
                name='conv_1')
h_pool1=max_pool_2d(h_conv1,
                    p_h=2,p_w=2,
                    s_h=2,s_w=2,
                    padding='SAME',name='pool_1')

h_conv2=conv_2d(h_pool1,
                k_h=5,k_w=5,
                s_h=1,s_w=1,
                output_channel=64,name='conv_2')

h_pool2=max_pool_2d(h_conv2,
                    p_h=2,p_w=2,s_h=2,s_w=2,
                    padding='SAME',name='pool_2')

part_4_shape=h_pool2.get_shape()
flatten_shape=part_4_shape[1]*part_4_shape[2]*part_4_shape[3]
flattened=tf.reshape(h_pool2,(-1,flatten_shape))

dense_1=dense_2d(flattened,output_dim=1024,
                 keep_prob=keep_prob,name='dense_1')
dense_2=dense_2d(dense_1,output_dim=10,
                 act=tf.nn.softmax,
                 keep_prob=1.0,name='dense_2')

loss=-tf.reduce_sum(y_input*tf.log(dense_2))
trainer=tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

In [23]:
'''
计算精度
'''
temp_acc=tf.equal(tf.argmax(dense_2,1),tf.argmax(y_input,1))
accuracy=tf.reduce_mean(tf.cast(temp_acc,'float'))

In [24]:
'''
train
'''
feature_map_dir='./feature_maps/'
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
   
    for i in range(20000):
        batch_x,batch_y=mnist.train.next_batch(50)
        
        if i%100==0:
            result,feature_map1=sess.run([accuracy,h_pool1],feed_dict={x_input:batch_x,
                                                y_input:batch_y,
                                                keep_prob:1.0})
            
            print(int(i/100),result,feature_map1.shape)
#             print(feature_map1[0,:,:,0])
            for j in range(32):
                cv2.imwrite('./feature_maps/img_'+str(j)+'.jpg',
                            feature_map1[24,:,:,j]*255)

        sess.run([trainer],feed_dict={x_input:batch_x,
                                    y_input:batch_y,
                                    keep_prob:0.5})

    batch_test_x=mnist.test.images
    batch_test_y=mnist.test.labels
    all_test=sess.run(accuracy,feed_dict={x_input:batch_test_x,
                                          y_input:batch_test_y,
                                          keep_prob:1.0})
    print(all_test)

0 0.06 (50, 14, 14, 32)
1 0.98 (50, 14, 14, 32)
2 0.94 (50, 14, 14, 32)


KeyboardInterrupt: 

In [17]:
cv2.imwrite??